# SHACL

This document contains a short introduction to [RDF](https://www.w3.org/TR/rdf11-primer/) using [rudof](https://rudof-project.github.io/).


## Preliminaries: Install and configure rudof

The library is available as `pyrudof`.

In [1]:
# @title
!pip install pyrudof
from pyrudof import Rudof, RudofConfig
rudof = Rudof(RudofConfig())

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 6.4 MB/s eta 0:00:00


## Validate using SHACL

In [9]:
# @title
rudof = Rudof(RudofConfig())

Let's read some RDF data.

In [10]:
rudof.read_data_str("""
prefix :       <http://example.org/>
prefix xsd:    <http://www.w3.org/2001/XMLSchema#>
prefix rdf:    <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix schema: <http://schema.org/>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>

:timbl  rdf:type    :Human ;
        :name       "Tim Berners-Lee" ;
        :birthPlace :london ;
        :birthDate  "1955-06-08"^^xsd:date ;
        :employer   :CERN ;
        :knows      _:1 .
:london rdf:type    :city, :metropolis ;
        :country    :UK .
:CERN   rdf:type    :Organization .
_:1     :birthPlace :Spain .
""")

In [11]:
rudof.read_shacl_str("""
prefix :       <http://example.org/>
prefix sh:     <http://www.w3.org/ns/shacl#>
prefix xsd:    <http://www.w3.org/2001/XMLSchema#>
prefix rdfs:   <http://www.w3.org/2000/01/rdf-schema#>
prefix schema: <http://schema.org/>

:Person a sh:NodeShape ;
 sh:targetClass :Human ;
 sh:property [ sh:path :name ;
               sh:datatype xsd:string;
               sh:minCount 1 ; sh:maxCount 1 ] ;
 sh:property [ sh:path :birthDate ; sh:datatype xsd:date;
                                    sh:maxCount 1 ] ;
 sh:property [ sh:path :birthPlace ;
               sh:node :Place; sh:maxCount 1 ] ;
 sh:property [
    sh:path :employer ;
    sh:node :Organization
] .

:Place a sh:NodeShape .
:Organization a sh:NodeShape .
""")

In [12]:
result = rudof.validate_shacl()

In [13]:
print(result.show_as_table())

No Errors found


The previous data didn't have errors. We can add some data with errors like the following:

In [14]:
rudof.read_data_str("""
prefix :       <http://example.org/>
prefix xsd:    <http://www.w3.org/2001/XMLSchema#>
prefix rdf:    <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix schema: <http://schema.org/>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>

:wrong1 rdf:type    :Human ;
        :name       "John" ;
        :birthDate  "1955-06-08"^^xsd:date, "1956-06-08"^^xsd:date .
:wrong2 rdf:type    :Human ;
        :name       23 ;
        :birthDate  "1955-06-08"^^xsd:date .
""", merge = True)

In [15]:
result = rudof.validate_shacl()
print(result.show_as_table())

╭──────────────┬─────────┬─────────────┬─────────────┬───────┬────────────────────────────────────╮
│ Severity     │ node    │ Component   │ Path        │ value │ Source shape                       │
├──────────────┼─────────┼─────────────┼─────────────┼───────┼────────────────────────────────────┤
│ sh:Violation │ :wrong1 │ sh:maxCount │ :birthDate, │       │ _:a2be1ef3e3525520e4d8385beb068da0 │
├──────────────┼─────────┼─────────────┼─────────────┼───────┼────────────────────────────────────┤
│ sh:Violation │ :wrong2 │ sh:datatype │ :name,      │ 23    │ _:edc7efa4afe26f7c1b229a48ad7e99a9 │
╰──────────────┴─────────┴─────────────┴─────────────┴───────┴────────────────────────────────────╯

